In [1]:
!pip install transformers datasets python-docx accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.7 MB/s eta 0:00:00


In [3]:
# prepare_dataset.py
from docx import Document
from typing import List, Dict

LABEL2ID = {
    "Запрос информации/документов": 0,
    "Официальная жалоба или претензия": 1,
    "Регуляторный запрос": 2,
    "Партнёрское предложение": 3,
    "Запрос на согласование": 4,
    "Уведомление или информирование": 5,
}


def load_examples_from_docx(path: str) -> List[Dict]:
    doc = Document(path)
    current_label = None
    samples = []

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue

        # Если абзац совпадает с одним из названий классов -> меняем текущую метку
        if text in LABEL2ID:
            current_label = text
            continue

        # Иначе считаем это текстом письма для текущего класса
        if current_label is not None:
            samples.append({
                "text": text,
                "label": LABEL2ID[current_label]
            })

    return samples


if __name__ == "__main__":
    path = "/content/letters.docx"  # путь в Colab/локально
    data = load_examples_from_docx(path)
    print(f"Найдено примеров: {len(data)}")
    for ex in data[:5]:
        print(ex["label"], ex["text"][:80], "...")



Найдено примеров: 54
0 Настоящим уведомляем о грубом нарушении условий договора №БС-1456 от 15.03.2023: ...
1 Настоящим уведомляем о грубом нарушении условий договора №БС-1456 от 15.03.2023: ...
2 На основании п. 4.2 Указания Банка России №55-У от 10.04.2024 просим представить ...
3 Добрый день! Предлагаем установить партнёрство в рамках совместного запуска цифр ...
3 Уважаемый Иванов Иван Иванович, ...


In [4]:
%%writefile prepare_dataset.py
from docx import Document

LABEL2ID = {
    "Запрос информации/документов": 0,
    "Официальная жалоба или претензия": 1,
    "Регуляторный запрос": 2,
    "Партнёрское предложение": 3,
    "Запрос на согласование": 4,
    "Уведомление или информирование": 5,
}

ID2LABEL = {v: k for k, v in LABEL2ID.items()}


def load_examples_from_docx(path: str):
    doc = Document(path)
    current_label = None
    samples = []

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue

        # Если текст совпадает с названием класса — это новый тип письма
        if text in LABEL2ID:
            current_label = text
            continue

        # Если это пример — добавляем
        if current_label is not None:
            samples.append({"text": text, "label": LABEL2ID[current_label]})

    return samples


Writing prepare_dataset.py


In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [6]:
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch
from prepare_dataset import load_examples_from_docx, LABEL2ID, ID2LABEL

BASE_MODEL_NAME = "DeepPavlov/rubert-base-cased"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def main():
    # 1. Загружаем данные из DOCX
    samples = load_examples_from_docx("/content/letters.docx")  # или своё имя файла

    print("Всего примеров:", len(samples))
    # 2. Превращаем в HuggingFace Dataset
    dataset = Dataset.from_list(samples)

    # 3. Разбиваем на train/val
    dataset = dataset.train_test_split(test_size=0.2, seed=42)
    train_ds = dataset["train"]
    val_ds = dataset["test"]

    # 4. Токенизатор и модель
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(
        BASE_MODEL_NAME,
        num_labels=len(LABEL2ID),
        id2label=ID2LABEL,
        label2id=LABEL2ID,
    ).to(DEVICE)

    def tokenize_batch(batch):
        return tokenizer(
            batch["text"],
            truncation=True,
            max_length=256,
            padding="max_length"
        )

    train_ds = train_ds.map(tokenize_batch, batched=True)
    val_ds = val_ds.map(tokenize_batch, batched=True)

    train_ds = train_ds.remove_columns(["text"])
    val_ds = val_ds.remove_columns(["text"])

    train_ds.set_format("torch")
    val_ds.set_format("torch")

    # 5. Аргументы обучения — только то, что точно есть в старых версиях
    args = TrainingArguments(
        output_dir="./rubert-letter-classifier",
        num_train_epochs=4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=2e-5,
        weight_decay=0.01,
        logging_steps=10,
        save_steps=500,
        seed=42,
    )

    # 6. Метрика вручную (accuracy)
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = logits.argmax(axis=-1)
        acc = (preds == labels).mean()
        return {"accuracy": float(acc)}

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
    )

    # 7. Обучаем
    trainer.train()

    # 8. Оцениваем после обучения (раз нет evaluation_strategy)
    print(trainer.evaluate())

    # 9. Сохраняем дообученную модель локально
    save_path = "./rubert-letter-classifier-final"
    trainer.save_model(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Модель сохранена в {save_path}")


if __name__ == "__main__":
    main()

Всего примеров: 54


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
10,1.638000
20,1.503200


{'eval_loss': 1.5046263933181763, 'eval_accuracy': 0.5454545454545454, 'eval_runtime': 0.2496, 'eval_samples_per_second': 44.064, 'eval_steps_per_second': 8.012, 'epoch': 4.0}
Модель сохранена в ./rubert-letter-classifier-final


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_PATH = "/content/rubert-letter-classifier-final"  # твоя обученная модель
BASE_MODEL = "DeepPavlov/rubert-base-cased"             # базовый токенизатор

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(DEVICE)

ID2LABEL = model.config.id2label  # мы задавали при обучении

def classify_letter(text: str):
    inputs = tokenizer(
        text,
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors="pt"
    ).to(DEVICE)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=-1)[0].cpu().numpy()

    best_idx = int(probs.argmax())
    return {
        "type": ID2LABEL[best_idx],
        "confidence": float(probs[best_idx]),
        "probs": {ID2LABEL[i]: float(p) for i, p in enumerate(probs)}
    }



# Пробуем на примере
text = """
Добрый день! Предлагаем установить партнёрство в рамках совместного запуска цифровой платформы для МСБ. Прикрепляем презентацию и условия сотрудничества. Готовы обсудить детали.
"""

res = classify_letter(text)
print(res)


{'type': 'Партнёрское предложение', 'confidence': 0.24981902539730072, 'probs': {'Запрос информации/документов': 0.11318337172269821, 'Официальная жалоба или претензия': 0.10759253054857254, 'Регуляторный запрос': 0.1215166300535202, 'Партнёрское предложение': 0.24981902539730072, 'Запрос на согласование': 0.17806315422058105, 'Уведомление или информирование': 0.22982530295848846}}


# GigaChat

In [9]:
!pip install -q "transformers>=4.45.0" accelerate sentencepiece

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "ai-sage/GigaChat3-10B-A1.8B-bf16"

# Определяем устройство
if torch.cuda.is_available():
    device_map = "auto"
    torch_dtype = torch.bfloat16
else:
    device_map = {"": "cpu"}
    torch_dtype = torch.float32

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch_dtype,
    device_map=device_map,
)

model.generation_config = GenerationConfig.from_pretrained(model_name)

print("Модель и токенизатор загружены.")
print("CUDA доступна:", torch.cuda.is_available())



tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00005-of-00010.safetensors:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/37.5k [00:00<?, ?B/s]

model-00008-of-00010.safetensors:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [4]:
SYSTEM_PROMPT = """
Ты — сотрудник службы клиентской поддержки крупного банка.
Ты отвечаешь на входящие письма клиентов, партнёров и регуляторов.

Требования к ответу:
- Стиль: официальный деловой, вежливый, без эмоций и просторечий.
- Не придумывай факты о продуктах, тарифах, договорах и нормативных актах.
- Если каких-то данных явно нет в письме, пиши, что требуется дополнительная информация
  или что вопрос будет передан в профильное подразделение.
- Соблюдай нейтральный, корректный тон даже при жалобах и претензиях.
- Ответ должен быть готовым текстом письма, который можно отправить от имени Банка.
"""

def build_messages(email_text: str, category: str | None = None):
    """
    Формирует список сообщений в формате chat-template для GigaChat.
    category — опционально: тип письма (жалоба, запрос информации, регуляторный запрос и т.п.)
    """
    user_instruction = "Ниже приведён текст письма, на которое нужно ответить от имени Банка.\n"
    if category:
        user_instruction += f"Тип письма: {category}.\n"
    user_instruction += (
        "Сформируй официальный ответ, подходящий для отправки клиенту/контрагенту.\n\n"
        f"Текст письма:\n\"\"\"\n{email_text}\n\"\"\""
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT.strip()},
        {"role": "user", "content": user_instruction},
    ]
    return messages


In [5]:
def gigachat_reply(
    email_text: str,
    category: str | None = None,
    max_new_tokens: int = 400,
    temperature: float = 0.2,
    top_p: float = 0.9,
):
    """
    Генерирует ответ на письмо от имени Банка с помощью GigaChat3-10B-A1.8B.
    :param email_text: текст входящего письма
    :param category: тип письма (например: "Официальная жалоба", "Регуляторный запрос", "Партнёрское предложение")
    """

    messages = build_messages(email_text, category=category)

    # Используем chat template из модели (как в оф. примере) :contentReference[oaicite:2]{index=2}
    input_tensor = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    )

    input_tensor = input_tensor.to(model.device)

    outputs = model.generate(
        input_tensor,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Отрезаем промпт и декодируем только сгенерированный ответ
    generated = outputs[0][input_tensor.shape[1]:]
    text = tokenizer.decode(generated, skip_special_tokens=True)

    # Немного подчистим
    return text.strip()


In [6]:
example_email = """
Добрый день! Предлагаем установить партнёрство в рамках совместного запуска цифровой платформы для МСБ. Прикрепляем презентацию и условия сотрудничества. Готовы обсудить детали.
"""

reply = gigachat_reply(
    email_text=example_email,
    category="Официальная жалоба или претензия",
    max_new_tokens=300,
    temperature=0.15,
)

print("=== Сгенерированный ответ Банка ===\n")
print(reply)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


=== Сгенерированный ответ Банка ===

Уважаемый [Имя клиента],

Сообщаем, что Ваше обращение по факту списания денежных средств со счета было рассмотрено. В ходе проверки установлено следующее:

В соответствии с условиями Договора банковского обслуживания № БС-1456 от 15.03.2023, списание средств возможно только после информирования клиента о предстоящей операции. Однако, как следует из предоставленных Вами документов, данное требование не было соблюдено банком.

На основании вышеизложенного, мы приняли решение вернуть сумму списанных средств в полном объеме. Ожидаем поступления денежных средств на Ваш счет до [указать дату].

Если у Вас возникнут дополнительные вопросы, просим обращаться в службу поддержки по телефону [номер телефона] или электронной почте [адрес электронной почты].


# YandexGPT

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-instruct"

# Определяем устройство
if torch.cuda.is_available():
    device_map = "auto"     # пусть accelerate сам раскидает
    torch_dtype = "auto"    # модель сама подберёт (F16)
else:
    device_map = {"": "cpu"}
    torch_dtype = "auto"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map=device_map,
    torch_dtype=torch_dtype,
)

# Можно подтянуть дефолтный generation_config, если понадобится
try:
    model.generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
except Exception:
    pass

print("Модель и токенизатор загружены.")
print("CUDA доступна:", torch.cuda.is_available())
print("Device map:", device_map)


In [ ]:
SYSTEM_PROMPT = """
Ты — сотрудник службы клиентской поддержки крупного банка.
Ты отвечаешь на входящие письма клиентов, партнёров и регуляторов.

Требования к ответу:
- Стиль: официальный деловой, вежливый, без эмоций и просторечий.
- Не придумывай факты о продуктах, тарифах, договорах и нормативных актах.
- Если каких-то данных явно нет в письме, напиши, что требуется дополнительная информация
  или что вопрос будет передан в профильное подразделение.
- Соблюдай нейтральный, корректный тон даже при жалобах и претензиях.
- Ответ должен быть готовым текстом письма, который можно отправить от имени Банка.
"""

def build_messages(email_text: str, category: str | None = None):
    """
    Формирует список сообщений для chat-template YandexGPT-5-Lite-8B-instruct.
    category — необязательно: тип письма (жалоба, регуляторный запрос и т.п.).
    """
    user_instruction = "Ниже приведён текст письма, на которое нужно ответить от имени Банка.\n"
    if category:
        user_instruction += f"Тип письма: {category}.\n"
    user_instruction += (
        "Сформируй официальный ответ, подходящий для отправки клиенту/контрагенту.\n\n"
        f"Текст письма:\n\"\"\"\n{email_text}\n\"\"\""
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT.strip()},
        {"role": "user", "content": user_instruction},
    ]
    return messages


In [ ]:
def yagpt_reply(
    email_text: str,
    category: str | None = None,
    max_new_tokens: int = 400,
    temperature: float = 0.2,
    top_p: float = 0.9,
):
    """
    Генерирует официальный ответ на письмо от имени Банка с помощью YandexGPT-5-Lite-8B-instruct.
    :param email_text: текст входящего письма
    :param category: тип письма (например: "Официальная жалоба", "Регуляторный запрос")
    """

    messages = build_messages(email_text, category=category)

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,  # добавляем промпт для генерации
        return_tensors="pt",
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Отрезаем prompt и берём только сгенерированный ответ
    generated = outputs[0][input_ids.shape[1]:]
    text = tokenizer.decode(generated, skip_special_tokens=True)

    return text.strip()


In [ ]:
example_email = """
Добрый день! Предлагаем установить партнёрство в рамках совместного запуска цифровой платформы для МСБ. Прикрепляем презентацию и условия сотрудничества. Готовы обсудить детали.
"""

reply = yagpt_reply(
    email_text=example_email,
    category="Официальная жалоба или претензия",
    max_new_tokens=300,
    temperature=0.15,
)

print("=== Сгенерированный ответ Банка (YandexGPT-5-Lite-8B-instruct) ===\n")
print(reply)
